# Is your house worth it's weight in gold?

Now we've calculated how much a given house rose in value over time - the question is, would you have been better off buying gold?

In the `data` folder, we have the daily price of one ounce of gold in USD, called `XAUUSD` in trading terms (e`X`change rate of `AU` aka Gold to `USD`)

We want to find out what would have happened if instead of buying a house, we would have bought an equivalent sum of gold. Finally we want to compare what the average profit would be if everyone had invested in gold instead of buying a house.

For simplicity, use the daily Closing price.

```{note}
Since gold is priced in USD, and our houseprices are in GBP, there is also a USD/GBP daily prices in the `data/fx` folder to convert between currencies.
```

Breaking it down:

1. Load daily gold prices into Iceberg - for bonus points, apply a yearly partitioning scheme
2. Load exchange rates into Iceberg - for bonus points, apply a yearly partitioning scheme
3. Convert daily gold prices from USD to GBP
4. For each row in our `profits` table, calculate the equivalent amount of gold they would have been able to purchase on that date
5. Calculate the total value of that amount of gold on the sell date
6. Calculate the profit of our gold trade
7. Compare to Gold profit with Housing profit

In [1]:
import polars as pl
from utils import catalog, engine
from pyiceberg.schema import Schema, NestedField
from pyiceberg.types import DecimalType, DateType, StringType
from pyiceberg.partitioning import PartitionSpec, YearTransform, PartitionField
from IPython.display import display
pl.Config.set_thousands_separator(",")

polars.config.Config

## 1. Load daily gold prices into Iceberg
Start by creating the table in Iceberg. To organize things a bit better, I'm creating a new namespace `commodities` - we could imagine putting a table for oil prices or silver in here

In [2]:
catalog.create_namespace_if_not_exists("commodities")

gold_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "price",
        DecimalType(precision=38, scale=2),
        required=True,
        doc="Price in USD of one ounce of gold",
    ),
    identifier_field_ids=[1],
)

In [3]:
gold_prices_t = catalog.create_table_if_not_exists(
    "commodities.gold",
    schema=gold_schema,
    partition_spec=PartitionSpec(
        PartitionField(
            source_id=1, field_id=1, transform=YearTransform(), name="date_year"
        )
    ),
)

Next, read in the CSV, picking out the two columns we care about, remembering to convert to the correct schema, 

In [4]:
gold_prices = (
    pl.scan_csv("data/gold/daily_gold_prices.csv", separator=";", try_parse_dates=True)
    .select(pl.col("Date").alias("date"), pl.col("Close").alias("price"))
    .collect()
)
gold_prices

date,price
datetime[μs],f64
2004-06-11 00:00:00,384.1
2004-06-14 00:00:00,382.8
2004-06-15 00:00:00,388.6
2004-06-16 00:00:00,383.8
2004-06-17 00:00:00,387.6
…,…
2025-01-28 00:00:00,"2,763.17"
2025-01-29 00:00:00,"2,759.68"
2025-01-30 00:00:00,"2,794.06"


In [5]:
gold_prices_t.append(gold_prices.to_arrow().cast(gold_schema.as_arrow()))

# 2. Load daily exchange rates into Iceberg

Similar process - create a new namespace and load the fx rates. Since this is technically a table of currency pairs, we can make the schema a bit more future-proof

In [6]:
catalog.create_namespace_if_not_exists("fx")

fx_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "from",
        StringType(),
        required=True,
        doc="From currency code of the currency pair",
    ),
    NestedField(
        3,
        "to",
        StringType(),
        required=True,
        doc="To currency code of the currency pair",
    ),
    NestedField(
        4,
        "exchange_rate",
        DecimalType(precision=38, scale=4),
        required=True,
        doc="Exchange rate of the currency pair",
    ),
    identifier_field_ids=[1, 2, 3],
)

Next, reading in the CSV file and transforming slightly to conform to our schema

In [7]:
fx_table = catalog.create_table_if_not_exists("fx.rates", schema=fx_schema)

In [8]:
fx_rates = (
    pl.scan_csv("data/fx/USD_GBP.csv", try_parse_dates=True)
    .with_columns(pl.lit("USD").alias("from"), pl.lit("GBP").alias("to"))
    .select(
        pl.col("date"),
        pl.col("from"),
        pl.col("to"),
        pl.col("close").cast(pl.Decimal(38, 4)).alias("exchange_rate"),
    )
    .collect()
)
fx_rates

date,from,to,exchange_rate
date,str,str,"decimal[38,4]"
2025-05-16,"""USD""","""GBP""",0.7529
2025-05-15,"""USD""","""GBP""",0.7516
2025-05-14,"""USD""","""GBP""",0.7540
2025-05-13,"""USD""","""GBP""",0.7516
2025-05-12,"""USD""","""GBP""",0.7590
…,…,…,…
2006-03-07,"""USD""","""GBP""",0.5760
2006-03-06,"""USD""","""GBP""",0.5713
2006-03-03,"""USD""","""GBP""",0.5696


In [9]:
fx_table.append(fx_rates.to_arrow().cast(fx_schema.as_arrow()))

## Business Logic
Next comes the business logic - the actual work of figuring out how much gold we could have bought and how much that would have sold for

In [10]:
sql = """
-- Convert USD gold prices into GBP denominated prices
with gold_prices as (
    select commodities.gold.date as gold_date, commodities.gold.price * fx.rates.exchange_rate as gold_price
    from commodities.gold
    join fx.rates on fx.rates.date = commodities.gold.date
), gold_purchase as (
-- Calculate how much gold (in ounces) the house owner could have bought at the time of purchase
    select address_id, 
    first_price / gold_price as purchased_gold
    from housing.profits
    join gold_prices on gold_date = housing.profits.first_day
), gold_sell as (
-- Calculate how much that amount of gold is worth at the time of sale, as well as the profit
    select housing.profits.address_id,
    cast((purchased_gold * gold_price) - first_price as DECIMAL(38, 2)) as gold_profit,
    cast(profit as DECIMAL(38, 2)) as house_profit
    from housing.profits
    join gold_purchase on gold_purchase.address_id = housing.profits.address_id
    join gold_prices on gold_prices.gold_date = last_day
)
select * from gold_sell
"""

In [11]:
gold_vs_house_profits = pl.read_database(sql, engine)
gold_vs_house_profits

address_id,gold_profit,house_profit
str,"decimal[38,2]","decimal[38,2]"
"""E7212FC4DD4DE6EEACEF3A2E069ACC…","40,230.82","-22,500.00"
"""C93F0B6F6F82401AFF1C511FF7ACEF…","15,498.03","39,500.00"
"""CE92C6E47E7B015B3A80A50C31AC79…","111,327.95","85,000.00"
"""996AD2EF5EFB730D42202B131BEB84…","77,923.33","200,000.00"
"""970B72DCDC9B719CDDD531E88D0692…","112,377.28",0.00
…,…,…
"""B62B37D0BB8169956985F389FDBCB3…","978,363.27","21,005.00"
"""C3A799B599BBF3E2C748FFE4F5BDE4…","3,201,682.77","9,754,579.00"
"""A8ABDF6D094837170D9C8C89D7CE6E…","83,729.77","2,050.00"


Now we have a per-address calculation, let's summarize the results

In [12]:
gold_vs_house_profits.select(pl.all().exclude("address_id")).describe()

statistic,gold_profit,house_profit
str,f64,f64
"""count""",1.164999e6,1.164999e6
"""null_count""",0.0,0.0
"""mean""","137,382.087434","63,217.298158"
"""std""","187,251.512417","112,895.232608"
"""min""","-918,104.38",-1.198e7
"""25%""","45,381.74","20,000.0"
"""50%""","92,987.92","45,000.0"
"""75%""","171,926.4","80,000.0"
"""max""","21,050,917.48",2.4992e7


How many percent would have done better buying gold than a house?

In [13]:
with pl.Config(set_tbl_rows=100):
    summary_df = (
        gold_vs_house_profits.select(
            pl.col("gold_profit")
            .sub(pl.col("house_profit"))
            .qcut(100, labels=[f"Q{i + 1}" for i in range(100)], include_breaks=True)
        )
        .unnest("gold_profit")
        .unique()
        .sort("breakpoint")
    )
    display(summary_df)

breakpoint,category
f64,cat
"-187,710.8238","""Q1"""
"-124,899.5792","""Q2"""
"-96,878.7716","""Q3"""
"-79,911.3604","""Q4"""
"-68,322.479","""Q5"""
"-59,485.292","""Q6"""
"-52,535.4192","""Q7"""
"-46,643.1372","""Q8"""
"-41,579.8236","""Q9"""


# Exercise: What does it look like in your region?

This is the picture for all of the UK - what does it look like for your region?